<a href="https://colab.research.google.com/github/minh-chaudang/IntroAI/blob/main/Bloxorz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [145]:
from copy import copy, deepcopy
import time
import os
from random import choice, choices, randint, randrange, random
import numpy as np
from math import floor

In [182]:
# Create a dictionary to save memory later
move    = {None: "Initial Block", 0 : "NONE", -1 : "UP", 1 : "DOWN", -2 : "LEFT", 2 : "RIGHT"}
state   = {0 : "STANDING", 1 : "LYING HORIZONTALLY", 2 : "LYING VERTICALLY", 3 : "SPLIT"}
square  = {0 : "NONE",              1 : "WEAK",                   2 : "STRONG", 
          3 : "TOGGLE HARD SWITCH", 4 : "UNOFFABLE HARD SWITCH",
          5 : "TOGGLE SOFT SWICH",  6 : "UNOFFABLE SOFT SWITCH",
          7 : "TELEPORT"}
on      = {0 : "OFF", 1 : "ON"}

In [183]:
class Block:
  def __init__(self, parts, last_move = None, parent = None):
    self.parts = sorted(parts, key = lambda x : 1000*x[0] + x[1])
    self.state = self.get_state()
    self.last_move = last_move
    self.parent = parent

  def get_state(self):
    row_diff = abs(self.parts[0][0] - self.parts[1][0])
    col_diff = abs(self.parts[0][1] - self.parts[1][1])

    if row_diff == 0 and col_diff == 0: return 0
    elif row_diff == 0 and col_diff == 1: return 1
    elif row_diff == 1 and col_diff == 0: return 2
    else: return 3
  
  def move(self, instruction, movable_part = None):
    if instruction == 0: return copy.copy(self)

    child_parts = deepcopy(self.parts)
    # Standing
    if self.state == 0:
      if instruction == -1:
        child_parts[0][0] -= 2
        child_parts[1][0] -= 1
      elif instruction == 1:
        child_parts[0][0] += 1
        child_parts[1][0] += 2
      elif instruction == -2:
        child_parts[0][1] -= 2
        child_parts[1][1] -= 1
      else:
        child_parts[0][1] += 1
        child_parts[1][1] += 2
    # Lying horizontally
    elif self.state == 1:
      if instruction == -1:
        child_parts[0][0] -= 1
        child_parts[1][0] -= 1
      elif instruction == 1:
        child_parts[0][0] += 1
        child_parts[1][0] += 1
      elif instruction == -2:
        child_parts[0][1] -= 1
        child_parts[1][1] -= 2
      else:
        child_parts[0][1] += 2
        child_parts[1][1] += 1
    # Lying vertically
    elif self.state == 2:
      if instruction == -1:
        child_parts[0][0] -= 1
        child_parts[1][0] -= 2
      elif instruction == 1:
        child_parts[0][0] += 2
        child_parts[1][0] += 1
      elif instruction == -2:
        child_parts[0][1] -= 1
        child_parts[1][1] -= 1
      else:
        child_parts[0][1] += 1
        child_parts[1][1] += 1
    #Split
    elif self.state == 3: child_parts[movable_part][abs(instruction) - 1] += np.sign(instruction)

    if self.state == 3: return Block(child_parts, (move[instruction], movable_part), self)
    else: return Block(child_parts, move[instruction], self)

  def __eq__(self, other):
    return self.parts == other.parts

In [186]:
class Special_square:
  def __init__ (self, name, position, adds = [], removes = [], on = 0):
    self.name = name
    self.position = position
    self.adds = adds
    self.removes = removes
    self.on = on

  def toggle(self):
    self.on = 1 - self.on
  
  def print(self):
    # Weak switch
    if self.name == 3: 
      if self.on == 1: print("Weak switch at", self.position, "is now on, squares", self.affects, "are added!")
      else: print("Weak switch at", self.position, "is now off, squares", self.affects, "are removed!")
    if self.name == 4:
      if self.on == 1: print("Strong switch at", self.position, "is now on, squares", self.affects, "are added!")
      else: print("Strong switch at", self.position, "is now off, squares", self.affects, "are removed!")
    if self.name == 5: print("Teleport at", self.position, "splits blocks into", self.affects[0], "and", self.affects[1])

In [187]:
class Map: 
  # A map stores a 2d-array of squares and its special squares
  def __init__(self, land, specials = None):
    self.land = np.array(land)
    self.row = len(land)
    self.col = len(land[0])
    self.specials = specials

    # Position of goal
    self.goal = np.argwhere(self.land == 10)[0]

  def __eq__(self, other):
    return (self.land == other.land).all()

  # Find a special square at a postion
  def find_special(self, position):
    return next(x for x in self.specials if x.position == position)

  # Get value of pos on map's length
  def get_value(self, pos):
    return self.land[pos[0]][pos[1]]

  # Check if a block is valid on this map
  def valid_block(self, block):
    # Position not in map, return False
    if not (block.parts[0][0] in range(self.row) and block.parts[1][0] in range(self.row) and block.parts[0][1] in range(self.col) and block.parts[1][1] in range(self.col)): 
      return False
    # One part on 0-square, return False
    if self.get_value(block.parts[0]) <= 0 or self.get_value(block.parts[1]) <= 0: return False
    # Standing on weak square, return False
    if block.state == 0 and self.get_value(block.parts[0]) < 2: return False

    return True

  # Check if two postions are connected
  # def is_connected(self, pos1, pos2):
  #   # Take all columns between them
  #   cols = self.land[pos1[1]:pos2[1]]

  # Calculate distance from this block to goal
  def distance_to_goal(self, block):
    cost1 = abs(block.parts[0][0] - self.goal[0]) + abs(block.parts[0][1] - self.goal[0])
    cost2 = abs(block.parts[1][0] - self.goal[0]) + abs(block.parts[1][1] - self.goal[0])
    return max(cost1, cost2)

  # Check if a block activates some special squares on this map and return theirs position
  def activated(self, block):
    activated = []
    # Standing on hard ones
    if block.state == 0:
      # Toggle hard switch
      if self.get_value(block.parts[0]) == 3 and block.parent.parts[0] != block.parts[0]: 
        activated.append(deepcopy(block.parts[0]))
      # Unoffable hard switch currently off
      if self.get_value(block.parts[0]) == 4 and block.parent.parts[0] != block.parts[0]:
        unoff_hard = self.find_special(block.parts[0])
        if unoff_hard.on == 0: activated.append(deepcopy(block.parts[0]))
      # Teleport
      if self.get_value(block.parts[0]) == 7 and block.parent.parts[0] != block.parts[0]:
        activated.append(deepcopy(block.parts[0]))
    # One parts on soft ones
    else:
      # Toggle soft switch
      if self.get_value(block.parts[0]) == 5 and block.parent.parts[0] != block.parts[0]: 
        activated.append(deepcopy(block.parts[0]))
      if self.get_value(block.parts[1]) == 5 and block.parent.parts[1] != block.parts[1]: 
        activated.append(deepcopy(block.parts[1]))
      # Unoffable soft switch currently off
      if self.get_value(block.parts[0]) == 6 and block.parent.parts[0] != block.parts[0]:
        unoff_soft = self.find_special(block.parts[0])
        if unoff_soft.on == 0: activated.append(deepcopy(block.parts[0]))
      if self.get_value(block.parts[1]) == 6 and block.parent.parts[0] != block.parts[1]:
        unoff_soft = self.find_special(block.parts[1])
    return activated

In [194]:
class State:
  def __init__ (self, map, block, parent = None):
    self.map = map
    self.block = block
    self.parent = parent

  def is_goal(self):
    return self.block.state == 0 and self.map.get_value(self.block.parts[0]) == 10

  def move(self, instruction, movable_part = None): 
    child_block = self.block.move(instruction, movable_part)
    if not self.map.valid_block(child_block): return None

    child_map = self.map
    activated = child_map.activated(child_block)

    if len(activated) > 0:
      child_map = deepcopy(self.map)
      for pos in activated:
        special = child_map.find_special(pos)
        # Activate switches
        if special.name in range(3, 7):
          if special.on == 0:
            special.on == 1
            for pos in special.adds:
              child_map.land[pos[0]][pos[1]] = 2
            for pos in special.removes:
              child_map.land[pos[0]][pos[1]] = 0
          else:
            special.on == 0
            for pos in special.adds:
              child_map.land[pos[0]][pos[1]] = 0
            for pos in special.removes:
              child_map.land[pos[0]][pos[1]] = 2
        else:
          child_block = Block(special.affects, move[instruction], self.block)
    return State(child_map, child_block, self)


  def expand(self):
    children = []
    if self.block.state == 3:
      for i in [-2, -1, 2, 1]:
        child0 = self.move(i, 0)
        if child0 is not None: children.append(child0)
        child1 = self.move(i, 1)
        if child1 is not None: children.append(child1)
    else:
      for i in [-2, -1, 2, 1]:
        child = self.move(i)
        if child is not None: children.append(child)
    return children

  def __eq__(self, other):
    return self.map == other.map and self.block == other.block

In [189]:
class Chromosome:
  def __init__(self, length):
    # Every gen is a move (if the block is unsplit, ignore the latter movable_part)
    self.length = length
    self.gens = choices([(-1,0), (-1,1), (1,0), (1,1), (2,0), (2,1), (-2,0), (-2,1)], k = length)
    self.last_gen = 0
    self.cost_to_goal = 1000
    self.last_state = None

In [195]:
class BLOXORZ:
  population_size = 20
  generation_limit = 10
  population = []
  best = []

  def __init__ (self, initial_state):
    self.initial_state = initial_state

  def DFShelper(self, stack, visited, loop, max_stack_size):
    while len(stack) > 0:
      max_stack_size = max(len(stack), max_stack_size)
      current = stack.pop()

      # Just to check if initial state is also goal
      if current.is_goal(): return current, loop, max_stack_size

      visited.append(current)
      children = current.expand()
      for child in children:
        if child.is_goal(): return child, loop, max_stack_size
        if child not in visited: 
          stack.append(child)
          # print(child.parent.block.parts, child.block.last_move, child.block.parts)
             
      loop += 1

    return None, loop, max_stack_size

  def DFS(self, state = None):
    if state is None: state = self.initial_state
    goal, loop, max_stack_size = self.DFShelper([state], [], 0, 0)
    if goal is None: print("No solution")
    else:
      path = [goal]
      while path[-1].parent is not None: path.append(path[-1].parent)
      path.reverse()
      print("DFS executed after", loop, "loops", "with max stack size", max_stack_size)
      for i in range(len(path)): print("Step", i, ":", path[i].block.last_move, path[i].block.parts)

  ###############################################
  ########## GENETIC ALGORITHM BEGINS!!! ########
  ###############################################

  # Calculate cost to goal of current moving procedure
  def cost_to_goal(self, chrom: Chromosome):
    current = self.initial_state
    for i in range(chrom.length):
      parent = current
      current = current.move(chrom.gens[i][0], chrom.gens[i][1])
      if current is None:
        chrom.last_gen = i-1
        chrom.cost_to_goal = parent.map.distance_to_goal(parent.block)
        chrom.last_state = parent
        return chrom.cost_to_goal
      else:
        chrom.last_gen = i
        chrom.cost_to_goal = current.map.distance_to_goal(current.block)
        chrom.last_state = current
        if chrom.cost_to_goal == 0: return 0

        # Very likely to get the goal
        if chrom.cost_to_goal < 3: 
          return chrom.cost_to_goal

    return chrom.cost_to_goal

  # Initiate a popolation
  def initPop(self):
    pop = []
    while (len(pop) < self.population_size):
      indi = Chromosome(randrange(self.initial_state.map.land.size//2, 2*self.initial_state.map.land.size))
      self.cost_to_goal(indi)
      # If the procedure can move more than 5 steps
      if indi.last_gen > 5: pop.append(indi)

    self.population = pop
    self.best = pop[0:5]

  def select_parents(self):
    cost_weights = np.ones(len(self.population))
    for i in range(self.population_size):
      cost = self.population[i].cost_to_goal
      if cost > 0: cost_weights[i] = 1/cost

    return choices(population = self.population, weights = cost_weights, k = 2)

  def cross_over(self, dad: Chromosome, mom: Chromosome):
    if dad == mom: return dad, mom

    length = min(len(dad.gens), len(mom.gens))
    index = randint(0, length)

    gen1 = dad.gens[0:index-1] + mom.gens[index:len(mom.gens)-1]
    gen2 = mom.gens[0:index-1] + dad.gens[index:len(mom.gens)-1]

    dad.gens = gen1
    dad.length = len(gen1)
    mom.gens = gen2
    mom.length = len(gen2)

    return dad,mom

  def mutate(chrom: Chromosome, prob = 0.5):
    for i in range(len(chrom.gens)):
      rand = random()

      if (rand > prob): 
        chrom.gens[i] = choice([(-1,0), (-1,1), (1,0), (1,1), (2,0), (2,1), (-2,0), (-2,1)])
    return 0

  def evovle(self):
    # Let 5 best indivuduals continue existing (without crossing)
    self.best += self.population[0:5]
    self.best = sorted(self.best, key = lambda indi: self.cost_to_goal(indi))
    self.best = self.best[0:5]
    # Let 5 best individuals continue living
    next_gen = deepcopy(self.population[0:5])
    for i in range(floor(len(self.population)/2)):
      parents = self.select_parents()
      child1, child2 = self.cross_over(parents[0], parents[1])
      next_gen += [child1, child2]
    
    self.population = next_gen
    

  def GA(self):
    self.initPop()
    
    for i in range(self.generation_limit):
      self.population = sorted(self.population, key = lambda indi: self.cost_to_goal(indi))
      if self.population[0].cost_to_goal == 0: return self.population[0].gens
      self.evovle()

    return self.best[0].cost_to_goal

In [199]:
level1 = BLOXORZ(State(Map([[2,2,2,0,0,0,0,0,0,0],
                            [2,2,2,2,2,2,0,0,0,0],
                            [2,2,2,2,2,2,2,2,2,0],
                            [0,2,2,2,2,2,2,2,2,2], 
                            [0,0,0,0,0,2,2,10,2,2], 
                            [0,0,0,0,0,0,2,2,2,0]]), 
                             Block([[1,1],[1,1]])))
level2 = BLOXORZ(State(Map([[0,0,0,0,0,0,2,2,2,2,0,0,2,2,2],
                            [2,2,2,2,0,0,2,2,3,2,0,0,2,10,2],
                            [2,2,5,2,0,0,2,2,2,2,0,0,2,2,2],
                            [2,2,2,2,0,0,2,2,2,2,0,0,2,2,2],
                            [2,2,2,2,0,0,2,2,2,2,0,0,2,2,2],
                            [2,2,2,2,0,0,2,2,2,2,0,0,0,0,0]], 
                            [Special_square(3, [2,2], adds = [[4,4], [4,5]]), Special_square(4, [1,8], [[4,10], [4,11]])]),
                             Block([[3,1],[3,1]])))
level3 = BLOXORZ(State(Map([[0,0,0,0,0,0,2,2,2,2,2,2,2,0,0],
                            [2,2,2,2,0,0,2,2,2,0,0,2,2,0,0],
                            [2,2,2,2,2,2,2,2,2,0,0,2,2,10,2],
                            [2,2,2,2,0,0,0,0,0,0,0,2,2,2,2],
                            [0,0,0,0,0,0,0,0,0,0,0,0,2,2,2]]),
                             Block([[2,1],[2,1]])))
level4 = BLOXORZ(State(Map([[0,0,0,1,1,1,1,1,1,1,0,0,0,0],
                            [0,0,0,1,1,1,1,1,1,1,0,0,0,0],
                            [2,2,2,2,0,0,0,0,0,2,2,2,0,0],
                            [2,2,2,0,0,0,0,0,0,0,2,2,0,0],
                            [2,2,2,0,0,0,0,0,0,0,2,2,0,0],
                            [2,2,2,0,0,2,2,2,2,1,1,1,1,1],
                            [2,2,2,0,0,2,2,2,2,1,1,1,1,1],
                            [0,0,0,0,0,2,10,2,0,0,1,1,2,1],
                            [0,0,0,0,0,2,2,2,0,0,1,1,1,1]]),
                             Block([[5,1],[5,1]])))
level5 = BLOXORZ(State(Map([[0,0,0,0,0,0,0,0,0,0,0,2,2,2,2],
                            [0,2,2,2,2,2,2,2,6,2,2,2,2,2,2],
                            [0,2,2,2,2,0,0,0,0,0,0,0,2,2,2],
                            [0,2,2,7,2,0,0,0,0,0,0,0,0,0,0],
                            [0,2,2,2,2,0,0,0,0,0,0,0,0,0,0],
                            [0,0,0,2,2,2,8,2,2,2,2,2,2,0,0],
                            [0,0,0,0,0,0,0,0,0,0,2,2,2,2,6],
                            [2,2,2,0,0,0,0,0,0,0,2,2,2,2,2],
                            [2,10,2,2,2,2,2,2,2,2,2,2,2,0,0],
                            [2,2,2,2,0,0,0,0,0,0,0,0,0,0,0]],
                            [Special_square(3, [1,8], [[1,5], [1,6]]), Special_square(3, [3,3], [[8,5], [8,6]]), Special_square(3, [5,6], [[8,5], [8,6]]), Special_square(3, [6,14], [[8,5], [8,6]])]),
                             Block([[1,13],[1,13]])))
level6 = BLOXORZ(State(Map([[0,0,0,0,0,2,2,2,2,2,2,0,0,0,0],
                            [0,0,0,0,0,2,0,0,2,2,2,0,0,0,0],
                            [0,0,0,0,0,2,0,0,2,2,2,2,2,0,0],
                            [2,2,2,2,2,2,0,0,0,0,0,2,2,2,2],
                            [0,0,0,0,2,2,2,0,0,0,0,2,2,10,2],
                            [0,0,0,0,2,2,2,0,0,0,0,0,2,2,2],
                            [0,0,0,0,0,0,2,0,0,2,2,0,0,0,0],
                            [0,0,0,0,0,0,2,2,2,2,2,0,0,0,0],
                            [0,0,0,0,0,0,2,2,2,2,2,0,0,0,0],
                            [0,0,0,0,0,0,0,2,2,2,0,0,0,0,0]]),
                             Block([[3,0],[3,0]])))
level7 = BLOXORZ(State(Map([[0,0,0,0,0,0,0,0,2,2,2,2,0,0,0],
                            [0,0,0,0,0,0,0,0,2,2,2,2,0,0,0],
                            [2,2,2,0,0,0,0,0,2,0,0,2,2,2,2],
                            [2,2,2,2,2,2,2,2,2,0,0,0,2,10,2],
                            [2,2,2,0,0,0,0,2,2,3,0,0,2,2,2],
                            [2,2,2,0,0,0,0,2,2,2,0,0,2,2,2],
                            [0,2,2,0,0,0,0,2,0,0,0,0,0,0,0],
                            [0,0,2,2,2,2,2,2,0,0,0,0,0,0,0]], 
                            [Special_square(4, [4,9], [[6,3]])]),
                             Block([[4,1],[4,1]])))
level8 = BLOXORZ(State(Map([[0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                            [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                            [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                            [2,2,2,2,2,2,0,0,0,2,2,2,2,2,2],
                            [2,2,2,2,7,2,0,0,0,2,2,2,2,10,2],
                            [2,2,2,2,2,2,0,0,0,2,2,2,2,2,2],
                            [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                            [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0],
                            [0,0,0,0,0,0,0,0,0,2,2,2,0,0,0]], 
                            [Special_square(7, [4,4], [[1,10], [7,10]])]),
                             Block([[4,1],[4,1]])))
level9 = BLOXORZ(State(Map([[2,2,2,2,0,0,0,2,0,0,0,2,2,2,2],
                            [2,2,2,2,0,0,0,2,0,0,0,2,2,7,2],
                            [2,2,2,2,2,2,2,2,2,2,2,2,2,2,2],
                            [0,0,0,0,0,0,2,10,2,0,0,0,0,0,0],
                            [0,0,0,0,0,0,2,2,2,0,0,0,0,0,0]],
                            [Special_square(7, [1,13], [[1,2], [1,12]])]),
                             Block([[1,1],[1,1]])))

In [200]:
level4.DFS()

DFS executed after 92 loops with max stack size 23
Step 0 : None [[5, 1], [5, 1]]
Step 1 : UP [[3, 1], [4, 1]]
Step 2 : RIGHT [[3, 2], [4, 2]]
Step 3 : DOWN [[5, 2], [5, 2]]
Step 4 : LEFT [[5, 0], [5, 1]]
Step 5 : DOWN [[6, 0], [6, 1]]
Step 6 : RIGHT [[6, 2], [6, 2]]
Step 7 : UP [[4, 2], [5, 2]]
Step 8 : UP [[3, 2], [3, 2]]
Step 9 : LEFT [[3, 0], [3, 1]]
Step 10 : DOWN [[4, 0], [4, 1]]
Step 11 : RIGHT [[4, 2], [4, 2]]
Step 12 : DOWN [[5, 2], [6, 2]]
Step 13 : LEFT [[5, 1], [6, 1]]
Step 14 : UP [[4, 1], [4, 1]]
Step 15 : UP [[2, 1], [3, 1]]
Step 16 : LEFT [[2, 0], [3, 0]]
Step 17 : DOWN [[4, 0], [4, 0]]
Step 18 : RIGHT [[4, 1], [4, 2]]
Step 19 : DOWN [[5, 1], [5, 2]]
Step 20 : DOWN [[6, 1], [6, 2]]
Step 21 : LEFT [[6, 0], [6, 0]]
Step 22 : UP [[4, 0], [5, 0]]
Step 23 : UP [[3, 0], [3, 0]]
Step 24 : RIGHT [[3, 1], [3, 2]]
Step 25 : UP [[2, 1], [2, 2]]
Step 26 : RIGHT [[2, 3], [2, 3]]
Step 27 : UP [[0, 3], [1, 3]]
Step 28 : RIGHT [[0, 4], [1, 4]]
Step 29 : RIGHT [[0, 5], [1, 5]]
Step 30 :